In [1]:
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

In [37]:
from datetime import datetime, timedelta


def cal_std_day(befor_day):   
    x = datetime.now() - timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)


In [2]:
# 질문 구하기

# 전역변수

#####################################################################################
#####################################################################################
# 의사의 페이지수 구하기
# 의사수
url = 'https://kin.naver.com/people/expert/index.naver?type=DOCTOR'
res = requests.get(url, headers=headers)
bs4 = BeautifulSoup(res.text, 'html.parser')
doc_count = int(bs4.find('div',{'class':'list_summary'}).find('strong').text.replace('명',''))
page_count = round(doc_count/10)

#####################################################################################
#####################################################################################
# 의사별 이름, 파라미터 구하기

#####################################################################################
#####################################################################################
#####################################################################################
#####################################################################################
base_url = 'https://kin.naver.com/people/expert/index.naver?type=DOCTOR&page='
doc_name = []
doc_param = []
for i in tqdm_notebook(range(1,page_count+1)) :
    url = base_url + str(i)
    res = requests.get(url, headers=headers)
    bs4 = BeautifulSoup(res.text, 'html.parser')
    lis = bs4.find('ul',{'class':'pro_list'}).findAll('li')
    for li in lis :

        # 의사의 고유번호
        doc_param.append(li.find('dt').a['href'].split('=')[1])
        # 의사이름
        doc_name.append(li.find('h5').text)
df_doc = pd.DataFrame({'doc_num':doc_param,'doc_name':doc_name})

/var/folders/dn/p6q55ry15bs3kyb8qw78mrb00000gn/T/ipykernel_1581/2135511403.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1,page_count+1)) :


  0%|          | 0/505 [00:00<?, ?it/s]

In [8]:

df_doc['doc_num'][0]
url = base + df_doc['doc_num'][400]
res = requests.get(url, headers=headers)
bs4 = BeautifulSoup(res.text, 'html.parser')

In [54]:
base = 'https://kin.naver.com/userinfo/expert/index.naver?u='
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
cols = ['doc_id','doc_name','is_pro','doc_pos','naver_grade']

data = []

for param in tqdm_notebook(df_doc['doc_num']):
    rows = []
    url = base + param
    res = requests.get(url, headers=headers)
    bs4 = BeautifulSoup(res.text, 'html.parser')

    #의사 id => key
    rows.append(param)
    print(url)
    #의사이름
    try :
        rows.append(bs4.find('div',{'class':'profile_section2'}).find('dt').text)
        # 의사 분야
        try : 
            doc_dpt=bs4.find('div',{'class':'profile_section2'}).find('span',{'class':'p_doctor'}).text.split(' ')
            # 전문의 여부 
            if doc_dpt[1] == '전문의':
                rows.append(1)
            else :
                rows.append(0)
        except : 
            doc_dpt=bs4.find('div',{'class':'profile_section2'}).find('span',{'class':'p_doctor'}).text
            if doc_dpt == '전문의':
                rows.append(1)
            else :
                rows.append(0)
        # 의사 근무지 및 직책
        try:
            doc_pos = bs4.find('div',{'class':'profile_section2'}).find('p',{'class':'position'}).text
            if doc_pos[-1] == ' ' :
                rows.append(doc_pos[:-1])
            else :
                rows.append(doc_pos)
        except :
            rows.append(None)
        try :
            # 지식인 등급
            rows.append(bs4.find('div',{'class':'profile_section3'}).findAll('dd')[2].text)
            tmp = dict(zip(cols,rows))
            data.append(tmp)
        except:
            rows.append(None)
            tmp = dict(zip(cols,rows))
            data.append(tmp)

    except :
        rows.append(None)
        rows.append(None)
        rows.append(None)
        rows.append(None)
        
res = {
    'meta':{
        'desc':'지식인 의사 정보',
        'cols':{
            'doc_id':'의사_파라미터',
            'doc_name': '의사이름',
            'is_pro': '전문의 여부 1:전문의 2:전문의 아니거나 치과의사',
            'doc_pos': '의사근무병원, 직책',
            'naver_grade' : '네이버지식인 등급'
        },
        'std_day':cal_std_day(0),

    },
    'data':data
}
res

/var/folders/dn/p6q55ry15bs3kyb8qw78mrb00000gn/T/ipykernel_1581/2409043462.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for param in tqdm_notebook(df_doc['doc_num']):


  0%|          | 0/5049 [00:00<?, ?it/s]

https://kin.naver.com/userinfo/expert/index.naver?u=vj9sWMlCerG%2BDT8H2SCiy2JKBBPAH95tW62%2F1Hxe6dM%3D
https://kin.naver.com/userinfo/expert/index.naver?u=vzwgR1au1P8UQHUK8hILTWcXd2uVNaY4JeOQkExePFg%3D
https://kin.naver.com/userinfo/expert/index.naver?u=ydXt%2Bzy3CbC9KLTQhFaE9VsNThG1tEzc3VWZdRPtyhM%3D
https://kin.naver.com/userinfo/expert/index.naver?u=CqVZt1NrTMDRS4UQ%2FbhNzpIDYzdNefZmR%2F4dxNs%2BSdo%3D
https://kin.naver.com/userinfo/expert/index.naver?u=ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D
https://kin.naver.com/userinfo/expert/index.naver?u=unqgzbt08jr5zIq37ftq0vea96T0JJ4GCVSHzJoimM8%3D
https://kin.naver.com/userinfo/expert/index.naver?u=GxMZ%2BUECvtZv4Lv79AqggSQI92rpZQJkYMtjcJ6C4QU%3D
https://kin.naver.com/userinfo/expert/index.naver?u=35VDCa0Xh%2BLepP607mmO0MqdR5iKsYM5vWqe8UPEheI%3D
https://kin.naver.com/userinfo/expert/index.naver?u=4ywHdz4uHO0ihVHclSQ07Fv2%2B12tCgSDCXKvP00ajNk%3D
https://kin.naver.com/userinfo/expert/index.naver?u=oQZQYui97%2BNgRcw7wHsECm%2FXJUVuQJT%2

https://kin.naver.com/userinfo/expert/index.naver?u=7RPofhVvRz%2F5HCoFVTbuvtPhiSe6zVyDfXLJRTHY%2FzM%3D
https://kin.naver.com/userinfo/expert/index.naver?u=H6hJEUKLumAW8vqGCj3dpHBWgYbIYX%2FefogpyJhb5TE%3D
https://kin.naver.com/userinfo/expert/index.naver?u=jNnYNl5%2FxSnOU%2Bws%2Bo%2Fg9cH3bNIfRKuwlzJRnGxTRsY%3D
https://kin.naver.com/userinfo/expert/index.naver?u=KIT3D7BInPxjK2iUXwJ%2BIpbENXIPW3Rk2NGdnJRWDyI%3D
https://kin.naver.com/userinfo/expert/index.naver?u=GNu9AZ8ZO1%2B6ZKG01o5Ym5zDrlZxiKXoClzQRvY6anM%3D
https://kin.naver.com/userinfo/expert/index.naver?u=8%2Baanfa49ZD2BPbIGn%2Bv%2FMu83bNTeD%2Bw2JjZpPDx40A%3D
https://kin.naver.com/userinfo/expert/index.naver?u=MBvRd4APMiGM2TyLncSnyo5mFt7eEM6FqsgVXtmtDxk%3D
https://kin.naver.com/userinfo/expert/index.naver?u=mO5JPZ%2FoR6MVFvzafDHKZHVzsAqUwtGZZEh969TIVBU%3D
https://kin.naver.com/userinfo/expert/index.naver?u=kk1wVwj9EYbYFNxIE2Y6Q3GKYsYCuGXcRJX7D1qxSw0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=fhqHHtrkxUZb2Tqdz%2FTam0PGU%2

https://kin.naver.com/userinfo/expert/index.naver?u=fDG%2Faa70I1ysIfyDBJgged%2FaJjBYz2pGRWjvpKFy0DQ%3D
https://kin.naver.com/userinfo/expert/index.naver?u=gaqUTyxX85F1B2pZXco3WUB5fjUUytOqsFMgv6%2F0nHc%3D
https://kin.naver.com/userinfo/expert/index.naver?u=ja99EtSwKaSd1jZ46hc0FRtC8kaOtPaXu85PX50LfmY%3D
https://kin.naver.com/userinfo/expert/index.naver?u=5CyOjE8uoWcak3tVgwvLqNZCpjft8GQCWXt6QebsQFw%3D
https://kin.naver.com/userinfo/expert/index.naver?u=ivN%2B5Asi7CP717cEauHD78CX79cbV0HYeF3J%2FMPe7s8%3D
https://kin.naver.com/userinfo/expert/index.naver?u=IvG4goy%2FLR8LNP8TD%2BJapRuD0hdf2NLIqsqIGrabMD4%3D
https://kin.naver.com/userinfo/expert/index.naver?u=0eTroy4NXt%2F3Pf9jVD%2B0Wqy%2BAdn3QsHLrz0Vx1Mx8gE%3D
https://kin.naver.com/userinfo/expert/index.naver?u=87Y24VvwtMpwvZiwwDjXh86JV2bEL6y9T559n9LvlRI%3D
https://kin.naver.com/userinfo/expert/index.naver?u=120Xe%2BYnrvCeEpTIXJSlAeLtp%2Bc3XqZxVlc9b910sa8%3D
https://kin.naver.com/userinfo/expert/index.naver?u=hxXPcPsn%2FaS%2FwF5RHnZCtPwQmy9%2

...

https://kin.naver.com/userinfo/expert/index.naver?u=%2BZqTVwxW9ylshzeZj8zqm8VeysPXdD5I7WgkJbo5vJ0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=W7P0dK4evWVziDVKnZ4%2F%2BczkjahIJsKE%2F6dAPtKAKA4%3D
https://kin.naver.com/userinfo/expert/index.naver?u=wqNGIVCJsUOY%2FnG8dVK%2FI5DhXU5PAFKHbx3CMyUnHR0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=ZLSWADvzRlLWEn9%2FOpfqRUHOZ%2Fd6q%2FvpYwbZQMk7V3E%3D
https://kin.naver.com/userinfo/expert/index.naver?u=qMJA5hiw4mgdNhf%2B%2BQ5MDQ3i73Ym5rjTCwgSlJ%2FnZG0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=RIwuMDmvL%2FPKcbhImiCND0Y6ebVNZK6jeIRzwcJfGk4%3D
https://kin.naver.com/userinfo/expert/index.naver?u=4LWLi%2FRUwbY5eJn9PKwP%2FfCOjyiN60WD9c9WLLVrKOw%3D
https://kin.naver.com/userinfo/expert/index.naver?u=SquaIGrIM7ZekYkZ%2FIitxJT38klmXTFkQOpyC4ip4o0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=IYl5OlaaFq40O3Q3IKUtlOyDiEHQaqkGI0A1vkF6dhY%3D
https://kin.naver.com/userinfo/expert/index.naver?u=WcWgKIyKSxzi76UwVvr3jfmoh

https://kin.naver.com/userinfo/expert/index.naver?u=6hoQoSGAEd9CX7WLE0Kv8L6Idbj%2Blynt06AfZ8GLlWg%3D
https://kin.naver.com/userinfo/expert/index.naver?u=tB2ctZXOxHFUwnHsYgp0N4VDtLf1U73sexk8BvIBy6E%3D
https://kin.naver.com/userinfo/expert/index.naver?u=t%2Fri1DdkOYdEZLEkXRYVSA611krBtbMlUTIotbMw3JU%3D
https://kin.naver.com/userinfo/expert/index.naver?u=qWoa84EaUzlqs7D%2BQgJeDy8BIMRewi3fnB9TfvqBiaU%3D
https://kin.naver.com/userinfo/expert/index.naver?u=SOc66NTBJOtd0YtSeYKk6oZUHC2mn4zJyYjUM9unrHU%3D
https://kin.naver.com/userinfo/expert/index.naver?u=O%2FyyQXmkQbOnlZScwYqrBYfDRqXiALcdX2wbVGbBUAk%3D
https://kin.naver.com/userinfo/expert/index.naver?u=O7DO3iM1bjSDy7SjqfJRWCe4wmosZ4M3CF1qYp8V9Cs%3D
https://kin.naver.com/userinfo/expert/index.naver?u=%2BxJzVVaOQgA8xRJYZ2J2REDCcV0NgNYVTvi3WfCQIYw%3D
https://kin.naver.com/userinfo/expert/index.naver?u=G0bKT%2BlmHaIcn2ly3dLTJnjsQ8fyZVpdMU%2FdKgjs9Cg%3D
https://kin.naver.com/userinfo/expert/index.naver?u=b%2B5OHbOIHdNbZBJt7AkpxjN3d2PP2NcEMoVrLEuEf

https://kin.naver.com/userinfo/expert/index.naver?u=%2BhKJfdNQGwsbArkZzp2pZcRN%2BdamJejacxBuQhgMab8%3D
https://kin.naver.com/userinfo/expert/index.naver?u=lpHZElYHw5S2I0M8oIqAPYbh%2BvOjNsGjv%2FZuP8Gh8Y0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=2I52kJa1vO2GAtUKkTjlsP7ib3Vfh9IJXUbJgi5qHAc%3D
https://kin.naver.com/userinfo/expert/index.naver?u=WmjZ%2F%2FfaIZY%2Fusr86HbGL3lHLdCtxDpwQv6uGm38IhI%3D
https://kin.naver.com/userinfo/expert/index.naver?u=lNNpdUpexhDFPjGulPb%2Fq2pDA%2BwiNtC0L0X%2B4xKeDTw%3D
https://kin.naver.com/userinfo/expert/index.naver?u=LrrKTZ7UfRWt2NXlgOnaMhiQIEVuS7GrJEakP9pcBZM%3D
https://kin.naver.com/userinfo/expert/index.naver?u=Yj4FgsurQ4A1xOT1aqWeHSOU2PRSS7KATdrfDjbz41c%3D
https://kin.naver.com/userinfo/expert/index.naver?u=TChlPrk2MudFoGPq6kNWcZlBeKt2ylt0PwUdTyP%2FEF0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=%2BwfexPqptBGW02CSdKxj%2FpGfQ7DIhRG9ARz4XbdGcFE%3D
https://kin.naver.com/userinfo/expert/index.naver?u=QCtyR5lGdLg%2BA3Y9y%2FIXBWZMW7o

https://kin.naver.com/userinfo/expert/index.naver?u=LxJ5gtWjIgK6HXqef1fuLKnXPqQgR%2BnOx0tpYppPRGU%3D
https://kin.naver.com/userinfo/expert/index.naver?u=7kPdgUCltuQChfKaGB%2FUnoyd3r%2B7NoNs2T%2BiGv8HxG4%3D
https://kin.naver.com/userinfo/expert/index.naver?u=IGZKAAPbqprH2zsIRgGrzJ7tNOhIbzx1wnEWsugS6Fc%3D
https://kin.naver.com/userinfo/expert/index.naver?u=bS0Bv0qR88sljATVG0MGnzpOcQJYnv9mngz3QWhiAO0%3D
https://kin.naver.com/userinfo/expert/index.naver?u=RnMUanYw8gChlSdmYAvTyA60evtY3jMOj%2BGDgHrFwKM%3D
https://kin.naver.com/userinfo/expert/index.naver?u=x3rDpgYNpp07Y6DK93rFkjvzUIboMn%2B9tc9pvWmoIFc%3D
https://kin.naver.com/userinfo/expert/index.naver?u=txobWJPIw%2FabeKwi3NNtgX4yz5ogN5c2dzk55FamjPo%3D
https://kin.naver.com/userinfo/expert/index.naver?u=XINVjkGPL0YngyVeu9eB8r%2Fm%2FMjDIT%2BYH0%2BAi7xg6WI%3D
https://kin.naver.com/userinfo/expert/index.naver?u=RQy0adornTpNbiyxsyZsYN8ciiNI4q1VfMrt7iXk9IE%3D
https://kin.naver.com/userinfo/expert/index.naver?u=46sPwT6YRNAo2KTlOXhhkhZ2PZfafRRtbxQ

{'meta': {'desc': '지식인 의사 정보',
  'cols': {'doc_id': '의사_파라미터',
   'doc_name': '의사이름',
   'is_pro': '전문의 여부 1:전문의 2:전문의 아니거나 치과의사',
   'doc_pos': '의사근무병원, 직책',
   'naver_grade': '네이버지식인 등급'},
  'std_day': '2022-10-21'},
 'data': [{'doc_id': 'vj9sWMlCerG%2BDT8H2SCiy2JKBBPAH95tW62%2F1Hxe6dM%3D',
   'doc_name': '윤충한',
   'is_pro': 1,
   'doc_pos': '윤충한정신건강의학과의원 원장',
   'naver_grade': '영웅'},
  {'doc_id': 'vzwgR1au1P8UQHUK8hILTWcXd2uVNaY4JeOQkExePFg%3D',
   'doc_name': '김무웅',
   'is_pro': 1,
   'doc_pos': '광주희망병원 원장',
   'naver_grade': '우주신'},
  {'doc_id': 'ydXt%2Bzy3CbC9KLTQhFaE9VsNThG1tEzc3VWZdRPtyhM%3D',
   'doc_name': '서동주',
   'is_pro': 1,
   'doc_pos': None,
   'naver_grade': '영웅'},
  {'doc_id': 'CqVZt1NrTMDRS4UQ%2FbhNzpIDYzdNefZmR%2F4dxNs%2BSdo%3D',
   'doc_name': '전평식',
   'is_pro': 1,
   'doc_pos': '시흥 행복한 의원 원장',
   'naver_grade': '수호신'},
  {'doc_id': 'ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D',
   'doc_name': '김철수',
   'is_pro': 1,
   'doc_pos': '삼육서울병원 전문의',
   'naver_grad

In [95]:

list(data[0].values())

['vj9sWMlCerG%2BDT8H2SCiy2JKBBPAH95tW62%2F1Hxe6dM%3D',
 '윤충한',
 1,
 '윤충한정신건강의학과의원 원장',
 '영웅']

In [98]:
dot_id = []
dot_name = []
dot_is_pro = []
dot_pos = []
dot_naver_grade = []
for i in range(0,len(data)):
    dot_id.append(list(data[i].values())[0])
    dot_name.append(list(data[i].values())[1])
    dot_is_pro.append(list(data[i].values())[2])
    dot_pos.append(list(data[i].values())[3])
    dot_naver_grade.append(list(data[i].values())[4])

    
df_doct = pd.DataFrame({'dot_id':dot_id,
                       'dot_name':dot_name,
                       'dot_is_pro':dot_is_pro,
                       'dot_pos':dot_pos,
                       'dot_naver_grade':dot_naver_grade})

In [100]:
df_doct.to_csv('지식인_답변_의사정보.csv', encoding='utf-8')

In [110]:
df_sam_1 = df_doct.iloc[:3000]
df_sam_2 = df_doct

## 의사정보 주 별로 업데이트 방법..
1. 신규의사
2. 기존의사 사라짐
    - 이 경우 의사테이블에서 사라질 필요는 없을 듯..
3. 의사정보 변경
    - 네이버 등급이 변경됬거나, dot_pos 의 소속 변경등

In [109]:
df_sam_1

,dot_id,dot_name,dot_is_pro,dot_pos,dot_naver_grade
0,vj9sWMlCerG%2BDT8H2SCiy2JKBBPAH95tW62%2F1Hxe6d...,윤충한,1,윤충한정신건강의학과의원 원장,영웅
1,vzwgR1au1P8UQHUK8hILTWcXd2uVNaY4JeOQkExePFg%3D,김무웅,1,광주희망병원 원장,우주신
2,ydXt%2Bzy3CbC9KLTQhFaE9VsNThG1tEzc3VWZdRPtyhM%3D,서동주,1,None,영웅
3,CqVZt1NrTMDRS4UQ%2FbhNzpIDYzdNefZmR%2F4dxNs%2B...,전평식,1,시흥 행복한 의원 원장,수호신
4,ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D,김철수,1,삼육서울병원 전문의,달신
...,...,...,...,...,...
2995,ShOHhJy57JCgOEGVEh3MAw71x%2Fu3KQIuQQ%2B8qYCeET...,문형진,1,비업의원 원장,고수
2996,4hP050dZH62wpPx6smy4DvgnoUYqruscsM%2BYbDFhpGo%3D,최규원,1,None,None
2997,m5A%2B4MZzGS8Xx9x2xycViDGKdktIOFLKsq0%2F%2BFNV...,이홍섭,1,None,고수
2998,MGC%2FojoeWzPNg5PgYmfoj62HOfYXsdWilEgE1iWQDRU%3D,이정호,1,잘보는안과의원 원장,고수


In [111]:
df_sam_2

,dot_id,dot_name,dot_is_pro,dot_pos,dot_naver_grade
0,vj9sWMlCerG%2BDT8H2SCiy2JKBBPAH95tW62%2F1Hxe6d...,윤충한,1,윤충한정신건강의학과의원 원장,영웅
1,vzwgR1au1P8UQHUK8hILTWcXd2uVNaY4JeOQkExePFg%3D,김무웅,1,광주희망병원 원장,우주신
2,ydXt%2Bzy3CbC9KLTQhFaE9VsNThG1tEzc3VWZdRPtyhM%3D,서동주,1,None,영웅
3,CqVZt1NrTMDRS4UQ%2FbhNzpIDYzdNefZmR%2F4dxNs%2B...,전평식,1,시흥 행복한 의원 원장,수호신
4,ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D,김철수,1,삼육서울병원 전문의,달신
...,...,...,...,...,...
5042,jU3DpyRxD33%2BorNChTGb30gQJN2nNDvylZs90evcwbQ%3D,김대호,1,알앤비성형외과의원 대표원장,None
5043,x8Vi%2BwJG2p%2FyNGkpWYd5JWNW8d6%2FGdtkQi8PEOwx...,이유선,0,서울클리어치과교정과치과의원 원장,None
5044,MFwcLBLKTyIXc%2BZbTI%2F%2B6%2FNZ7Y2TjKi5EKvpoN...,주성훈,0,None,None
5045,e8Hy%2B0hauV5hgJlQ7gIfVNJPPiCwvCLXJym4GUx7FZA%3D,김현도,1,온유정신건강의학과의원 원장,None


In [116]:
df_up.columns

Index(['dot_id', 'dot_name_x', 'dot_is_pro_x', 'dot_pos_x',
       'dot_naver_grade_x', 'dot_name_y', 'dot_is_pro_y', 'dot_pos_y',
       'dot_naver_grade_y'],
      dtype='object')

In [121]:
# 1. 의사정보 추가하기

# 기존의 데이터에 새로운 데이터를 outer 방식으로 조인, on은 dot_id 그렇다면..
# 기존에 없던 의사정보는 dot_name == null 일 것
df_up = df_sam_1.merge(df_sam_2,left_on='dot_id', right_on='dot_id',how='outer')
# 아래 df는 추가된 데이터만 추출 한것 
df_up_1 = df_up[pd.isna(df_up['dot_name_x'])]
df_up_1_end = df_up_1[['dot_id','dot_name_y', 'dot_is_pro_y', 'dot_pos_y',
       'dot_naver_grade_y']]
# 컬럼 재설정
df_up_1_end = df_up_1_end.rename(columns={'dot_name_y':'dot_name','dot_is_pro_y':'dot_is_pro',
                            'dot_pos_y':'dot_pos','dot_naver_grade_y':'dot_naver_grade'})

df_up_1_end_fin = pd.concat([df_sam_1,df_up_1_end])


In [142]:
# 그냥 outer로 조인한다음 새로운값으로 대체 해주면 아무문제없어보임?
# 만약에 없어진 의사는 어떡함?? 그렇다면.. 애초에 y값에 dot_id 값이 없었을 것임
df_up = df_sam_1.merge(df_sam_2,left_on='dot_id', right_on='dot_id',how='outer')
df_end = df_up[['dot_id','dot_name_y', 'dot_is_pro_y', 'dot_pos_y',
       'dot_naver_grade_y']].rename(columns={'dot_name_y':'dot_name','dot_is_pro_y':'dot_is_pro',
                            'dot_pos_y':'dot_pos','dot_naver_grade_y':'dot_naver_grade'})


In [128]:
df_sam_1[df_sam_1['dot_name']=='윤병우']

,dot_id,dot_name,dot_is_pro,dot_pos,dot_naver_grade


In [129]:
df_sam_1.loc[5047]=['sadfasdfsadf','윤병우',1,None,'영웅']

/var/folders/dn/p6q55ry15bs3kyb8qw78mrb00000gn/T/ipykernel_1581/181494600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sam_1.loc[5047]=['sadfasdfsadf','윤병우',1,None,'영웅']


In [143]:
df_up = df_sam_1.merge(df_sam_2,left_on='dot_id', right_on='dot_id',how='outer')
df_end_2 = df_up[(df_up['dot_name_x']=='윤병우') & (pd.isna(df_up['dot_name_y']))][['dot_id','dot_name_x', 'dot_is_pro_x',
                                                                            'dot_pos_x', 'dot_naver_grade_x']].rename(columns={'dot_name_x':'dot_name','dot_is_pro_x':'dot_is_pro',
                                    'dot_pos_x':'dot_pos','dot_naver_grade_x':'dot_naver_grade'})

In [170]:
df_fin = pd.concat([df_end,df_end_2]).reset_index().drop(columns='index')
# 의사이름 없는 행은 제거
df_fin = df_fin[~pd.isna(df_fin['dot_name'])].astype({'dot_is_pro':int})



,dot_id,dot_name,dot_is_pro,dot_pos,dot_naver_grade
0,vj9sWMlCerG%2BDT8H2SCiy2JKBBPAH95tW62%2F1Hxe6d...,윤충한,1,윤충한정신건강의학과의원 원장,영웅
1,vzwgR1au1P8UQHUK8hILTWcXd2uVNaY4JeOQkExePFg%3D,김무웅,1,광주희망병원 원장,우주신
2,ydXt%2Bzy3CbC9KLTQhFaE9VsNThG1tEzc3VWZdRPtyhM%3D,서동주,1,None,영웅
3,CqVZt1NrTMDRS4UQ%2FbhNzpIDYzdNefZmR%2F4dxNs%2B...,전평식,1,시흥 행복한 의원 원장,수호신
4,ZpLHwcZs6VN8eRB64%2FqBWJ45B1HSM8srkcTGsmusWwA%3D,김철수,1,삼육서울병원 전문의,달신
...,...,...,...,...,...
5044,x8Vi%2BwJG2p%2FyNGkpWYd5JWNW8d6%2FGdtkQi8PEOwx...,이유선,0,서울클리어치과교정과치과의원 원장,None
5045,MFwcLBLKTyIXc%2BZbTI%2F%2B6%2FNZ7Y2TjKi5EKvpoN...,주성훈,0,None,None
5046,e8Hy%2B0hauV5hgJlQ7gIfVNJPPiCwvCLXJym4GUx7FZA%3D,김현도,1,온유정신건강의학과의원 원장,None
5047,%2B9taThLbso9RMgrdYcXMeQ0hxLL2PE6iM4m0rhd94VY%3D,양우영,1,서울선혈관통증센터 봉직의,None
